<a href="https://colab.research.google.com/github/aleixopirez/Arquitectura-de-Big-Data/blob/main/analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código para analizar los medidores que son posibles candidatos a cambio por garantía.

In [ ]:
!pip install pandas
import pandas as pd
import os

In [ ]:
data = pd.read_excel("/content/dssGestionMayo.xlsx")

In [ ]:
nombre_carpeta = "Archivos"

path_carpeta = "/content/" + nombre_carpeta

os.makedirs(path_carpeta)

In [ ]:
# Ruta al archivo de Excel
archivo_excel = "/content/dssGestionMayo.xlsx"

# Leer el archivo de Excel en un DataFrame llamado "analisis"
analisis = pd.read_excel(archivo_excel)

# Eliminar las columnas especificadas dado a que la mayoría son datos con valor 0 y por que no contienen datos relevantes  para nuestro análisis
columnas_a_eliminar = ['id', 'param_time_before_nfc_off', 'tip_alarms', 'param_nfc_time_off', 'param_num_nst', 'param_nfc_time_off_nst', 'param_valve_test','est_med', 'dd_diagn', 'est_card', 'est_valvula']
analisis = analisis.drop(columns=columnas_a_eliminar, errors='ignore')

In [ ]:
# Verificar tipos de datos en las columnas, la columna fec_ult_ope debe ser datetime64, el resto deben ser int64
for c in analisis.columns:
    print(c, analisis[c].dtype)

serial_med int64
power_battery_life int64
measure_battery_life int64
fec_ult_ope datetime64[ns]
cont_act_valvula int64
cont_antifraud int64
cont_empty_card int64
cont_events int64
cont_valid_card int64
cont_invalid_card int64
lts_positive_cons int64


In [ ]:
# Convertir la columna 'fec_ult_ope' al formato DD/MM/AAAA debido a que no nos interesa las horas ni minutos, solo los dias, meses y años
analisis['fec_ult_ope'] = pd.to_datetime(analisis['fec_ult_ope']).dt.strftime('%d/%m/%Y')

# Contar la cantidad de filas antes de eliminar duplicados
filas_antes = len(analisis)

# Eliminar duplicados de la columna 'serial_med' porque los aplicativos envían varios registros del mismo medidor, en donde se repiten los mismos datos en todas las columnas
analisis = analisis.drop_duplicates(subset=['serial_med'])

# Contar la cantidad de filas después de eliminar duplicados
filas_despues = len(analisis)

# Calcular la cantidad de duplicados eliminados y la cantidad de valores únicos restantes
duplicados_eliminados = filas_antes - filas_despues
valores_unicos = filas_despues

# Ruta para guardar el archivo Excel modificado
archivo_excel_modificado = "/content/archivo_modificado.xlsx"

# Guardar el DataFrame modificado en un nuevo archivo de Excel
analisis.to_excel(archivo_excel_modificado, index=False)

print(f"Archivo guardado con éxito. Se eliminaron {duplicados_eliminados} duplicados, quedando {valores_unicos} valores únicos.")

# Ruta al archivo Excel modificado
archivo_modificado = "/content/archivo_modificado.xlsx"

Archivo guardado con éxito. Se eliminaron 8204 duplicados, quedando 32131 valores únicos.


In [ ]:
# Leer el archivo Excel modificado en un DataFrame
analisis_modificado = pd.read_excel(archivo_modificado)

# Filtrar el DataFrame para seleccionar solo las filas donde 'cont_act_valvula' >= 400
# ya que estos medidores tienen una alta probabilidad de que estén defectuosos, lo que ocasionaría de que la válvula quede abierta pasando agua.
filtro = analisis_modificado['cont_act_valvula'] >= 400
resultados_cont_val = analisis_modificado[filtro]

# Ruta para guardar el archivo con los resultados filtrados
archivo_resultados = "/content/Archivos/resultados_cont_val.xlsx"

# Guardar el DataFrame filtrado en un nuevo archivo Excel
resultados_cont_val.to_excel(archivo_resultados, index=False)

print("Archivo de resultados guardado con éxito.")

Archivo de resultados guardado con éxito.


In [ ]:
resultados_cont_val

,serial_med,power_battery_life,measure_battery_life,tip_alarms,fec_ult_ope,cont_act_valvula,cont_antifraud,cont_empty_card,cont_events,cont_valid_card,cont_invalid_card,lts_positive_cons
12,20201019925,70445,64888,20,07/05/2024,1152,0,267,1457,305,41,478820
21,20211023232,81673,71548,20,09/05/2024,586,0,181,765,167,42,159950
31,20201017787,73631,60731,34,09/05/2024,673,0,166,990,312,157,489160
32,20201019101,74867,61967,0,09/05/2024,616,0,537,993,366,73,327870
49,20191013344,81500,61220,20,08/02/2024,426,0,60,560,128,32,178840
...,...,...,...,...,...,...,...,...,...,...,...,...
32075,20191014987,77437,59135,0,02/05/2024,473,0,478,706,226,40,493660
32078,20211026784,86426,77922,20,07/05/2024,429,0,179,579,138,47,206610
32082,20211022567,75769,70770,20,07/05/2024,790,0,423,1075,271,341,187740
32097,20191014022,70232,57305,4,10/05/2024,997,0,536,1339,303,93,490130


In [ ]:
analisis

,serial_med,power_battery_life,measure_battery_life,tip_alarms,fec_ult_ope,cont_act_valvula,cont_antifraud,cont_empty_card,cont_events,cont_valid_card,cont_invalid_card,lts_positive_cons
0,2000000001,100000,100,0,21/12/2018,0,0,0,0,0,0,0
1,20181005131,95900,91396,820,06/05/2024,60,0,33,94,32,4,31530
2,20211027126,88816,78105,20,10/05/2024,295,0,19,423,114,16,143500
3,20211023178,93905,89896,20,26/04/2022,192,0,12,255,55,35,125650
4,20231032931,100000,100,0,10/01/2024,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
40325,20191010561,84831,56997,24,26/04/2024,65,0,171,173,98,23,148460
40326,20191017085,100000,100,0,17/01/2022,0,0,0,0,0,0,0
40330,20211023383,99507,98965,21,30/04/2024,3,0,0,9,4,0,160
40331,20191013886,99043,97186,20,28/04/2024,6,0,4,13,5,1,2290


In [ ]:
# Ruta al archivo Excel
archivo_excel = "/content/archivo_modificado.xlsx"

# Leer el archivo de Excel en un DataFrame
analisis = pd.read_excel(archivo_excel)

# Filtrar el DataFrame para seleccionar solo las filas donde 'power_battery_life' <= 45000
# este porcentaje de batería que significa un 45 % de ese porcentaje hacia abajo tiene alta probabilidad de que falle el lcd osea se apague el medidor.
filtro_bateria_baja = analisis['power_battery_life'] <= 45000
bateria_baja = analisis[filtro_bateria_baja]


# Ruta para guardar el archivo con los resultados de batería baja
archivo_bateria_baja = "/content/Archivos/low_battery.xlsx"

# Guardar el DataFrame filtrado en un nuevo archivo Excel
bateria_baja.to_excel(archivo_bateria_baja, index=False)

print("Archivos de batería baja guardado con éxito.")


Archivos de batería baja guardado con éxito.


In [ ]:
bateria_baja

,serial_med,power_battery_life,measure_battery_life,tip_alarms,fec_ult_ope,cont_act_valvula,cont_antifraud,cont_empty_card,cont_events,cont_valid_card,cont_invalid_card,lts_positive_cons
244,20181004187,20322,50900,20,16/04/2024,91,0,6,191,78,224,130710
364,20211022524,0,0,0,15/04/2024,0,0,0,0,0,0,0
588,20211025273,4524,77641,20,07/05/2024,85,0,51,146,52,17,191660
659,20191013702,39355,55815,24,02/05/2024,2784,0,113,3382,694,245,1056090
784,20171000340,0,0,0,15/04/2020,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
30718,20191006465,0,73657,00a0,28/06/2023,231,0,86,291,64,44,116210
30913,20191006908,38792,55757,20,01/04/2024,129,0,2,277,126,153,124680
31121,20231031692,0,63386,884,18/03/2024,4,0,15,30,10,0,2820
31585,20211024841,12558,80029,0,03/01/2024,2,0,7,54,40,4,75460


In [ ]:
# Ruta al archivo Excel
archivo_excel = "/content/archivo_modificado.xlsx"

# Leer el archivo de Excel en un DataFrame
analisis = pd.read_excel(archivo_excel)

# Filtrar el DataFrame para seleccionar solo las filas donde 'cont_antifraud' >= 5
# debido a que los valores iguales o superiores a 5 arrojan una alta probabilidad de que el medidor halla sido manipulado por lo tanto pierde la garantía
filtro_antifraude = analisis['cont_antifraud'] >= 5
antifraude = analisis[filtro_antifraude]

# Ruta para guardar el archivo con los resultados de antifraude
archivo_antifraude = "/content/Archivos/antifraud.xlsx"

# Guardar el DataFrame filtrado en un nuevo archivo Excel
antifraude.to_excel(archivo_antifraude, index=False)

print("Archivos de antifraude guardado con éxito.")


Archivos de antifraude guardado con éxito.


In [ ]:
antifraude

,serial_med,power_battery_life,measure_battery_life,tip_alarms,fec_ult_ope,cont_act_valvula,cont_antifraud,cont_empty_card,cont_events,cont_valid_card,cont_invalid_card,lts_positive_cons
5,20211025559,65249,80088,4a24,10/03/2024,64,17,54,133,30,2,32840
66,20181001586,78107,47638,800,05/04/2024,255,255,0,556,293,168,504020
90,20191017031,88536,66406,20,27/08/2023,90,90,57,142,44,8,96310
171,20191012125,98117,94966,20,31/01/2020,18,18,2,43,17,2,14370
399,20181003042,87595,65996,220,05/02/2024,92,12,42,161,34,29,52200
...,...,...,...,...,...,...,...,...,...,...,...,...
30347,20181004300,93850,85656,4,21/05/2023,13,13,3,30,13,1,17450
31116,20181004319,97800,93418,20,10/09/2020,6,6,3,12,7,1,1630
31189,20181003717,86815,68917,421,01/09/2022,194,122,0,582,118,76,130
31640,20201019444,83411,70644,0a34,21/05/2023,409,10,102,566,190,3,192830


In [ ]:
directorio = '/content/Archivos/'

dataframes = []


for archivo in os.listdir(directorio):
    if archivo.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(directorio, archivo))
        dataframes.append(df)


df_final = pd.concat(dataframes, ignore_index=True)


df_final.to_excel('/content/Unificado.xlsx', index=False)